In [1]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scipy.linalg as la
import numpy.random as npr
from itertools import combinations, chain
from numpy.linalg import inv
import sympy
# import rpy2.robjects as ro
# import pandas.rpy.common as com
# from rpy2.robjects.packages import importr

# base = importr('base')

In [2]:
def Y1_loglike(Y,G,Z,beta,gamma,sigma_11,q_j,n1):
    n = Y.shape[0]
    p = beta.shape[0]
    c = gamma.shape[0]
    j = len(q_j)
    
    sum_n1 = 0.0
    sum_n2 = 0.0
    sum_pdf = 0.0
    ind_gz = np.diag([1]*n1)
    m = n1
    
    for i in range(n1):
        mu_i = np.dot(G[i,:],beta) + np.dot(Z[i,:],gamma)
        sum_n1 += stats.norm.logpdf(Y[i,0], mu_i, np.sqrt(sigma_11)) + np.log(np.dot(q_j,ind_gz[:,i]))
    for k in range(n1,n):
        for j in range(m):
            mu_j = np.dot(G[j,:],beta) + np.dot(Z[j,:],gamma)
            sum_pdf += stats.norm.pdf(Y[k,0],mu_j,np.sqrt(sigma_11))*q_j[j]
        sum_n2 += np.log(sum_pdf)
    return(sum_n1 + sum_n2)

In [3]:
def EM_Y1(Y, G, Z, b_0, g_0, sigma_11_0, q_0, tol=1e-6, max_iter=1000):
    import scipy.stats as stats
    
    n = Y.shape[0]
    n1, p = G.shape
   
    # assume that m = n1 (m uniques pairs of observed values)
    gz_com = [zip(l,Z.tolist()) for l in combinations(G.tolist(),len(Z))]
    chain_gz = chain(*gz_com)
    unique_gz = list(chain_gz)
    m = len(unique_gz)
    if(m == n1):
        ind_gz = np.identity(m)      

    ll_old = 0.0
    b1 = b_0
    g1 = g_0
    sigma_11 = sigma_11_0
    q = q_0
    iters = 0
    for iters in range(max_iter):
        iters += 1
        ll_new = 0.0
        
        # E-step
        psi_n1 = np.identity(m)
        psi_n2 = []
        mus_m = np.dot(G,b1) + np.dot(Z,g1)
        for i in range(n1,n):
            numer = stats.norm.pdf(Y[i,0],mus_m,np.sqrt(sigma_11))
            denom = sum(numer)
            psi_n2 += [x/denom for x in numer]
        psi_n = np.vstack((psi_n1,np.array(psi_n2).reshape(((n-n1),m))))
        # M-step
        ### update b1 and g1
        W = np.vstack((G.T,Z.T))
        a11 = np.dot((G**2).T,psi_n.T).sum()
        a12 = np.dot(np.multiply(G,Z).T,psi_n.T).sum()
        a22 = np.dot((Z**2).T,psi_n.T).sum()
        eta_hat = np.dot(la.inv(np.array([[a11,a12],[a12,a22]])),np.dot(np.dot(W,psi_n.T),Y))
        b1 = eta_hat[0,:]
        g1 = eta_hat[1,:]

        ### update sigma_11
        a1 = (np.tile(Y,(1,m)) - np.tile(np.dot(eta_hat.T,W),(n,1)))**2
        sigma_11 = (1./n)*np.dot(a1.T,psi_n).sum()

        ### update q_j
        q = (1./n)*psi_n.sum(0)

        ### compute new log likelihood value
        ll_new = Y1_loglike(Y, G, Z, b1, g1, sigma_11, q, n1)
        if np.abs(ll_new - ll_old) < tol:
            break

        ll_old = ll_new
    Omega_1 = Y1_covar(Y, G, Z, b1, g1, sigma_11, q, psi_n, m)
    return(b1,g1,sigma_11,q,Omega_1)

In [4]:
def Y1_covar(Y, G, Z, b1, g1, sigma_11, q, psi_n, m):
    n = Y.shape[0]
    # symbolic equation 1st and 2nd derivatives of U
    b, g, s, q_s, y_s, G_s, Z_s = sympy.symbols('b g s q_s y_s G_s Z_s')
    U = sympy.symbols('U',cls = sympy.Function)
    U =  -0.5*sympy.log(s) - (0.5/s)*(y_s - b*G_s - g*Z_s)**2 + sympy.log(q_s)
    grad_sym = sympy.Matrix([b, g, s, q_s])
    l1 = [sympy.diff(U, sym) for sym in grad_sym]
    l2 = sympy.Matrix(sympy.hessian(U, grad_sym))
    f_l1 = sympy.lambdify((b, g, s, q_s, y_s, G_s, Z_s), l1, 'numpy')
    f_l2 = sympy.lambdify((b, g, s, q_s, y_s, G_s, Z_s), l2, 'numpy')

    fl1_qs = sympy.lambdify(q_s, l1[3], 'numpy')
    l1_qs = np.array(fl1_qs(q)).reshape((m,1))
    fl2_qs = sympy.lambdify(q_s, l2[3,3], 'numpy')
    l2_qs = np.append(np.zeros((3,m)),np.diag(fl2_qs(q)),axis=0)

    part1Q = 0.0
    part2Q = 0.0
    part4Q = 0.0
    for i in range(n):
        part3Q = 0.0
        for j in range(m):
            l2T = f_l2(b1,g1,sigma_11,q[j],Y[i,0],G[j,0],Z[j,0])[0:3,0:3]
            temp1 = psi_n[i,j]*np.column_stack((np.append(l2T,np.zeros((m,3)),axis=0),l2_qs))
            part1Q += temp1
            
            llT = np.vstack((np.array(f_l1(b1,g1,sigma_11,q[j],Y[i,0],G[j,0],Z[j,0])[0:3]),l1_qs))
            part2Q += psi_n[i,j]*np.dot(llT,llT.T)
            part3Q += psi_n[i,j]*llT
        part4Q += np.dot(part3Q,part3Q.T)
    Q_1 = -part1Q - part2Q + part4Q
    
    D = np.append((np.identity(m+2)),[[0,0,0]+[-1]*(m-1)],axis=0)
    F = np.dot(np.dot(D.T,Q_1),D)
    Omega_1 = la.inv(F)
    return(Omega_1)

In [5]:
def Y2_est(Y1, G1, Z1, b1, g1, Y2, G2, Z2, Omega_bg_1):
    import statsmodels.api as sm
    
    n2 = Y2.shape[0]
    Y1_hat = Y1 - np.dot(G1,b1) - np.dot(Z1,g1)
    X = np.column_stack((Y1_hat,G2,Z2))
    est = sm.OLS(Y2,X).fit()
    delta, b2, g2 = est.params
    sigma_22 = (1./n2)*((Y2 - delta*Y1_hat - np.dot(G2,b2) - np.dot(Z2,g2))**2).sum()
    Omega_2 = Y2_covar(delta,b2,g2,sigma_22,b1,g1,Y1,Y2,G1,G2,Z1,Z2,Omega_bg_1)
    
    return(delta, b2, g2, sigma_22, Omega_2)

In [6]:
def Y2_covar(delta, b2, g2, sigma_22, b1, g1, Y_1, Y_2, G_1, G_2, Z_1, Z_2, Omega_bg_1):
    import scipy.linalg as la
    
    Y1_hat = Y_1 - b1*G_1 - g1*Z_1
    Y1_G2 = np.dot(Y1_hat.T,G_2)
    Y1_Z2 = np.dot(Y1_hat.T,Z_2)
    G2_Z2 = np.dot(G_2.T,Z_2)
    G2_2 = np.dot(G_2.T,G_2)
    Z2_2 = np.dot(Z_2.T,Z_2)
    G1_Z1 = np.dot(G_1.T,Z_1)
    U = np.array([np.dot(Y1_hat.T,Y1_hat),Y1_G2,Y1_Z2,Y1_G2,G2_2,G2_Z2,Y1_Z2,G2_Z2,Z2_2]).reshape((3,3))
    V = np.array([np.dot(Y_2.T,Y1_hat),np.dot(Y_2.T,G_2),np.dot(Y_2.T,Z_2)]).reshape((3,1))
    dU_b1 = np.array([-2*G_1.sum()+2*b1*G2_2+2*g1*G1_Z1,-np.dot(G_1.T,G_2),-np.dot(G_1.T,Z_2),-np.dot(G_1.T,G_2),[0],[0],
                      -np.dot(G_1.T,Z_2),[0],[0]]).reshape((3,3))
    dV_b1 = np.array([-np.dot(G_1.T,Y_2),[0],[0]]).reshape((3,1))
    dU_g1 = np.array([-2*Z_1.sum()+2*g1*Z2_2+2*b1*G1_Z1,-np.dot(G_2.T,Z_1),-np.dot(Z_1.T,Z_2),
                      -np.dot(G_2.T,Z_1),[0],[0],-np.dot(Z_1.T,Z_2),[0],[0]]).reshape((3,3))
    dV_g1 = np.array([-np.dot(Z_1.T,Y_2),[0],[0]]).reshape((3,1))
    
    U_inv = la.inv(U)
    J = np.column_stack((-np.dot(np.dot(np.dot(U_inv,dU_b1),U_inv),V) + np.dot(U_inv,dV_b1),
              -np.dot(np.dot(np.dot(U_inv,dU_g1),U_inv),V) + np.dot(U_inv,dV_g1)))
    
    Omega_2 = sigma_22*U_inv + np.dot(np.dot(J,Omega_bg_1),J.T)
    return(Omega_2)

In [7]:
### test
npr.seed(123451)
n_test = 250
n1_test = 100
m_test = n1_test

Y_test = npr.normal(loc=3,scale=1,size=(n_test,1))
G_test = npr.binomial(2, 0.45, (n1_test,1))
Z_test = npr.normal(size=(n1_test,1))

beta = np.array([0.0])
gamma = np.array([0.0])
s_11 = np.var(Y_test)
q_j = np.repeat(np.array([1./m_test]),n1_test)

In [8]:
res = EM_Y1(Y_test, G_test, Z_test, beta, gamma, s_11, q_j)
# print Y_test.shape
# print G_test.shape
# print Z_test.shape
# b1 = beta
# g1 = gamma
# sigma_11 = s_11
# psi_n1 = np.identity(m_test)
# psi_n2 = []
# mus_m = np.dot(G_test,b1) + np.dot(Z_test,g1)
# print psi_n1.shape
# for i in range(n1_test,n_test):
#     numer = stats.norm.pdf(Y_test[i,0],mus_m,np.sqrt(sigma_11))
#     denom = sum(numer)
#     psi_n2 += [x/denom for x in numer]
# psi_n = np.vstack((psi_n1,np.array(psi_n2).reshape(((n_test-n1_test),m_test))))
# print psi_n.shape

In [9]:
b1 = res[0]
g1 = res[1]
sigma_11 = res[2]
q = res[3]
Omega_1 = res[4]

In [10]:
%timeit EM_Y1(Y_test, G_test, Z_test, beta, gamma, s_11, q_j)

1 loops, best of 3: 13 s per loop


In [11]:
psi_n1 = np.identity(m_test)
aa = np.empty((n_test-m_test,m_test))
aa.fill(1./m_test)
psi_n = np.vstack((psi_n1,aa))
%timeit Y1_covar(Y_test, G_test, Z_test, b1, g1, sigma_11, q, psi_n, m_test)

1 loops, best of 3: 6.61 s per loop


In [12]:
%load_ext line_profiler

In [13]:
stats = %prun -r -q Y1_covar(Y_test, G_test, Z_test, b1, g1, sigma_11, q, psi_n, m_test)
stats.sort_stats('time').print_stats(10);

          1229899 function calls (1227448 primitive calls) in 6.160 seconds

   Ordered by: internal time
   List reduced from 303 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.752    1.752    7.056    7.056 <ipython-input-4-24b752ed7942>:1(Y1_covar)
    25000    1.538    0.000    2.022    0.000 <string>:1(<lambda>)
    25252    0.935    0.000    0.935    0.000 {numpy.core._dotblas.dot}
    75002    0.488    0.000    0.566    0.000 {numpy.core.multiarray.concatenate}
   175005    0.373    0.000    0.373    0.000 {numpy.core.multiarray.array}
   125006    0.145    0.000    0.198    0.000 defmatrix.py:290(__array_finalize__)
    25000    0.134    0.000    0.484    0.000 defmatrix.py:244(__new__)
    50000    0.123    0.000    0.245    0.000 shape_base.py:60(atleast_2d)
   227042    0.110    0.000    0.110    0.000 {isinstance}
    25000    0.105    0.000    0.138    0.000 defmatrix.py:312(__getitem__)




In [14]:
lstats = %lprun -r -f Y1_covar Y1_covar(Y_test, G_test, Z_test, b1, g1, sigma_11, q, psi_n, m_test)
lstats.print_stats()

Timer unit: 1e-06 s

Total time: 8.05777 s
File: <ipython-input-4-24b752ed7942>
Function: Y1_covar at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Y1_covar(Y, G, Z, b1, g1, sigma_11, q, psi_n, m):
     2         1            7      7.0      0.0      n = Y.shape[0]
     3                                               # symbolic equation 1st and 2nd derivatives of U
     4         1          195    195.0      0.0      b, g, s, q_s, y_s, G_s, Z_s = sympy.symbols('b g s q_s y_s G_s Z_s')
     5         1           49     49.0      0.0      U = sympy.symbols('U',cls = sympy.Function)
     6         1          474    474.0      0.0      U =  -0.5*sympy.log(s) - (0.5/s)*(y_s - b*G_s - g*Z_s)**2 + sympy.log(q_s)
     7         1          105    105.0      0.0      grad_sym = sympy.Matrix([b, g, s, q_s])
     8         5         5676   1135.2      0.1      l1 = [sympy.diff(U, sym) for sym in grad_sym]
     9         

In [15]:
npr.seed(8471)
n2_test = 50
Y2_test = npr.normal(size=(n2_test,1))
G2_test = npr.binomial(2, 0.4, (n2_test,1))
Z2_test = npr.normal(size=(n2_test,1))

Y1 = np.array(Y_test[0:n2_test,0]).reshape((n2_test,1))
G1 = np.array(G_test[0:n2_test,0]).reshape((n2_test,1))
Z1 = np.array(Z_test[0:n2_test,0]).reshape((n2_test,1))
delta, b2, g2, sigma_22, Omega_2 = Y2_est(Y1, G1, Z1, b1.reshape((1,1)), g1.reshape((1,1)), Y2_test, G2_test, Z2_test, Omega_1[0:2,0:2])
print delta, b2, g2, sigma_22, Omega_2

-0.0319117545127 0.0798058234468 -0.0290625832643 0.928320487552 [[0.005381702453469957 -0.007075259662005475 -0.0002699862522480634]
 [-0.007075259662005475 0.03368741054853071 -0.0030843721611234043]
 [-0.0002699862522480635 -0.0030843721611234043 0.01576110926792826]]


In [16]:
%timeit Y2_est(Y1, G1, Z1, b1.reshape((1,1)), g1.reshape((1,1)), Y2_test, G2_test, Z2_test, Omega_1[0:2,0:2])

1000 loops, best of 3: 827 µs per loop


In [17]:
stats = %prun -r -q Y2_est(Y1, G1, Z1, b1.reshape((1,1)), g1.reshape((1,1)), Y2_test, G2_test, Z2_test, Omega_1[0:2,0:2])
stats.sort_stats('time').print_stats(10);

          298 function calls (297 primitive calls) in 0.002 seconds

   Ordered by: internal time
   List reduced from 93 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       37    0.000    0.000    0.000    0.000 {numpy.core._dotblas.dot}
        2    0.000    0.000    0.000    0.000 decomp_svd.py:15(svd)
        1    0.000    0.000    0.001    0.001 <ipython-input-6-f1bd5769cec5>:1(Y2_covar)
       13    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    0.000    0.000 linalg.py:1225(svd)
        1    0.000    0.000    0.002    0.002 <ipython-input-5-d636261f0aee>:1(Y2_est)
       23    0.000    0.000    0.000    0.000 {numpy.core.multiarray.array}
        1    0.000    0.000    0.000    0.000 _methods.py:77(_var)
        3    0.000    0.000    0.000    0.000 blas.py:172(find_best_blas_type)
        2    0.000    0.000    0.000    0.000 tools.py:374(rank)




In [18]:
lstats = %lprun -r -f Y2_est Y2_est(Y1, G1, Z1, b1.reshape((1,1)), g1.reshape((1,1)), Y2_test, G2_test, Z2_test, Omega_1[0:2,0:2])
lstats.print_stats()

Timer unit: 1e-06 s

Total time: 0.001536 s
File: <ipython-input-5-d636261f0aee>
Function: Y2_est at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Y2_est(Y1, G1, Z1, b1, g1, Y2, G2, Z2, Omega_bg_1):
     2         1            6      6.0      0.4      import statsmodels.api as sm
     3                                               
     4         1            3      3.0      0.2      n2 = Y2.shape[0]
     5         1           41     41.0      2.7      Y1_hat = Y1 - np.dot(G1,b1) - np.dot(Z1,g1)
     6         1           24     24.0      1.6      X = np.column_stack((Y1_hat,G2,Z2))
     7         1          999    999.0     65.0      est = sm.OLS(Y2,X).fit()
     8         1           33     33.0      2.1      delta, b2, g2 = est.params
     9         1           43     43.0      2.8      sigma_22 = (1./n2)*((Y2 - delta*Y1_hat - np.dot(G2,b2) - np.dot(Z2,g2))**2).sum()
    10         1          386    386.0

In [19]:
%timeit Y2_covar(delta, b2, g2, sigma_22, b1, g1, Y1, Y2_test, G1, G2_test, Z1, Z2_test, Omega_1[0:2,0:2])

1000 loops, best of 3: 292 µs per loop
